# Movie Scraping
This notebook allows you to retrieve data on movies (imdb ID, year, director, popularity, genre...) on IMDB database. You can get the movies from lists or directly from the imdb top. Also download cover of the movies from IMDB.

The obtained data are in notebooks/movizz/csv/metadata_movies.csv

In [ ]:
import urllib.parse
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
from datetime import datetime

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
FOLDER_IMG = "/home/tanguy/data/movizz/img"
FOLDER_CSV = "/home/tanguy/data/movizz/csv"

## Where to get movies ?

### Directly with the name of the movies

In [ ]:
# https://www.lefigaro.fr/cinema/2017/06/28/03002-20170628ARTFIG00180-les-100-meilleurs-films-de-tous-les-temps-selon-les-lecteurs-du-magazine-empire.php
list1 = ["Le Parrain", "Star Wars, épisode V: L'Empire contre-attaque", "The Dark Knight", "Les évadés", "Pulp Fiction", "Les Affranchis", "Les Aventuriers de l'arche perdue", "Les Dents de la mer", "Star Wars, épisode IV : Un nouvel espoir", "Le Seigneur des anneaux : La Communauté de l'anneau", "Retour vers le futur", "Le Parrain, 2e partie", "Blade Runner", "Alien", "Aliens", "Le Seigneur des anneaux: Le Retour du roi", "Fight Club de David Fincher", "Inception", "Jurrasic Park", "Die Hard, Piège de cristal", "2001: l'Odyssée de l'espace", "Apocalypse Now", "Le Seigneur des anneaux ", "Matrix", "Terminator 2: Le Jugement dernier", "Heat", "Le Bon, la Brute et le Truand", "Casablanca", "The Big Lebowski", "Seven", "Taxi Driver", "Usual Suspects", "La Liste de Schindler", "Les Gardiens de la Galaxie", "Shining", "Les Infiltrés", "The Thing", "Mad Max: Fury Road", "Il faut sauver le soldat Ryan", "12 hommes en colère", "Eternal Sunshine of the Spotless Mind", "There Will Be Blood", "Vol au-dessus d'un nid de coucou", "Gladiator", "Drive", "Citizen Kane", "Interstellar", "Le Silence des Agneaux", "Trainspotting", "Lawrence d'Arabie", "La vie est belle", "Il était une fois dans l'Ouest", "Psychose", "Sueurs Froides", "Le Labyrinthe de Pan", "Reservoir Dogs", "Whiplash", "Inglourious Basterds", "E.T.", "American Beauty", "Forrest Gump", "La La Land", "Donnie Darko", "L.A. Confidential", "Avengers", "Star Wars, épisode VI: Le Retour du Jedi", "Memento", "Ghostbusters", "Chantons sous la pluie", "Le Roi Lion", "Hot Fuzz", "Fenêtre sur Cour", "Les Sept Samouraïs", "Mulholland Drive", "Fargo", "Orange Mécanique", "Toy Story", "Old Boy", "Captain America: Civil War", "Le Voyage de Chihiro", "The Social Network", "Certains l'aiment chaud", "True Romance", "Rocky", "Léon", "Indiana Jones et la Dernière Croisade", "Predator", "L'Exorciste", "Shaun of the Dead", "No Country For Old Men", "Le Prestige", "Terminator", "Princess Bride", "Lost in Translation", "Premier Contact", "Will Hunting", "Titanic", "Le fabuleux destin d'Amélie Poulain", "Raging Bull", "Stand by me"]
# https://www.senscritique.com/liste/Les_meilleurs_films_de_tous_les_temps/1398987
list2 = ['Inception', 'Le Parrain', 'Forrest Gump', 'The Dark Knight - Le Chevalier noir', 'Le Prestige', 'Les Affranchis', 'Reservoir Dogs', 'The Dark Knight Rises', 'Le Retour du Jedi', 'Shutter Island', 'Star Wars : Épisode III - La Revanche des Sith', 'Il était une fois en Amérique', 'Apocalypse Now', 'Les Infiltrés', 'Seven', 'La Liste de Schindler', 'Le Parrain : 2e Partie', 'Le Bon, la Brute et le Truand', 'Django Unchained', "L'Empire contre-attaque", 'Le Loup de Wall Street', 'Pulp Fiction', 'Le Grand Bleu', 'Full Metal Jacket', 'Fight Club', 'La Ligne verte', 'American History X', 'Gran Torino', "Vol au-dessus d'un nid de coucou", 'Les Évadés', 'Heat', 'Indiana Jones et la Dernière Croisade', 'Whiplash', 'Léon', 'Dallas Buyers Club', 'Gladiator', 'La Guerre des Étoiles', 'Intouchables', "Voyage au bout de l'enfer", 'Ip Man : La Légende du grand maître', 'Avatar', 'Le Professionnel', 'Cent mille dollars au soleil', 'Titanic', 'Raging Bull', 'Interstellar', 'Inglourious Basterds', 'Taxi Driver', "L'Armée des ombres", 'Scarface', 'Jurassic Park', 'Kill Bill : Volume 1', "L'Impasse", 'Million Dollar Baby', 'Shining', 'Orange mécanique', 'Take Shelter', 'Mystic River', 'Le Vieux Fusil', 'Mommy', 'Usual Suspects', '12 hommes en colère', 'Retour vers le futur', "Les Aventuriers de l'arche perdue", 'Les Aventures de Rabbi Jacob', 'Alien : Le 8ème Passager', 'Prisoners', 'Il faut sauver le soldat Ryan', 'Star Wars : Épisode I - La Menace fantôme', 'Les Incorruptibles', 'Impitoyable', 'American Sniper', 'Braveheart', 'Les Tontons flingueurs', 'Pirates des Caraïbes : La Malédiction du Black Pearl', 'Le Seigneur des Anneaux : Le Retour du roi', 'Matrix', 'Duel', 'La Grande Vadrouille', 'Barry Lyndon', 'Gone Girl', 'Kingsman : Services secrets', 'Pour une poignée de dollars', 'Pirates des Caraïbes : La Fontaine de Jouvence', 'Le Cinquième Élément', 'Le Dictateur', "Pirates des Caraïbes : Jusqu'au bout du monde", 'Pirates des Caraïbes : Le Secret du coffre maudit', 'La vie est belle', 'Indiana Jones et le Temple maudit', "Le Seigneur des Anneaux : La Communauté de l'anneau", 'Birdman', 'The Revenant', "Le Discours d'un roi", "Star Wars : Épisode II - L'Attaque des clones", 'Rencontres du troisième type', 'Conjuring : Les Dossiers Warren', 'Le Seigneur des Anneaux : Les Deux Tours', 'La Rage au ventre', 'Le Parrain : 3e Partie', 'Conjuring 2 : Le Cas Enfield', 'Ali', 'Les Dents de la mer', 'Alien 3']
# https://www.imdb.com/chart/top/?sort=nv,desc&mode=simple&page=1
list3 = ['Les évadés','The Dark Knight: Le chevalier noir','Inception','Fight Club','Pulp Fiction','Forrest Gump','Matrix',"Le Seigneur des anneaux : La Communauté de l'anneau",'Le Seigneur des anneaux : Le Retour du roi','Le parrain','The Dark Knight Rises','Interstellar','Le Seigneur des anneaux : Les Deux Tours','Seven','Django Unchained','Gladiator','Batman Begins','Le Silence des agneaux','Inglourious Basterds','Il faut sauver le soldat Ryan','Star Wars: Épisode IV - Un nouvel espoir','La liste de Schindler','Les infiltrés','Le prestige','Le loup de Wall Street',"L'Empire contre-attaque",'La ligne verte','Le parrain, 2ème partie','Shutter Island','Memento','American Beauty','Retour vers le futur','American History X','Léon','V pour vendetta','Les affranchis','Kill Bill: Volume I','WALL·E','Terminator 2 : Le Jugement dernier','Usual Suspects','Braveheart','Le Monde de Nemo','Le Retour du Jedi','The Lion King','The Truman Show','Là-haut','Reservoir Dogs',"Vol au-dessus d'un nid de coucou",'Joker','Sixième sens','Eternal Sunshine of the Spotless Mind','Shining','Toy Story',"Les Aventuriers de l'arche perdue",'Mad Max: Fury Road','Jurassic Park','Will Hunting','Gone Girl',"No Country for Old Men - Non, ce pays n'est pas pour le vieil homme","Un homme d'exception",'Avengers: Infinity War','Arrête-moi si tu peux','Monstres & Cie','Terminator','Avengers: Endgame','Alien - Le 8ème passager','Piège de cristal','Snatch - Tu braques ou tu raques','Requiem for a Dream','Harry Potter et les reliques de la mort: Partie 2','Orange mécanique','Toy Story 3','Intouchables','Scarface','The Big Lebowski','Le pianiste','Taxi Driver','Gran Torino','Whiplash','The Grand Budapest hotel',"Le fabuleux destin d'Amélie Poulain",'La Cité de Dieu','Blade Runner','Indiana Jones et la dernière croisade','Le Bon, la brute, le truand','12 hommes en colère','Full Metal Jacket','Dragons','Aliens - Le retour','Le voyage de Chihiro','Logan','12 Years a Slave','Million Dollar Baby','Trainspotting','La vie est belle','Le labyrinthe de Pan','Fargo','Vice-versa','Apocalypse Now','Psychose',"2001: L'odyssée de l'espace",'Prisoners','Heat','Into the Wild','Arnaques, crimes et botanique','L.A. Confidential','Parasite','Casablanca','Rocky','There Will Be Blood','Old Boy','Monty Python : Sacré Graal !','Casino',"Dr. Folamour ou : Comment j'ai appris à ne plus m'en faire et à aimer la bombe !",'Fenêtre sur cour','Warrior','Rush','Tu ne tueras point','La couleur des sentiments','3 Billboards: Les panneaux de la vengeance','Le cercle des poètes disparus','Spotlight','1917','Citizen Kane','La vie est belle','Princess Bride','Platoon','Impitoyable','Coco','The Thing','Amadeus','Room','Monty Python: La Vie de Brian','Spider-Man: New Generation','Green Book: Sur les routes du Sud','Sueurs froides','Stand by me - Compte sur moi','La vie des autres','3 Idiots','Princesse Mononoké','Hôtel Rwanda','La chute','Le château ambulant','Raging Bull','Les sept samouraïs',"Voyage au bout de l'enfer",'Il était une fois en Amérique',"Il était une fois dans l'Ouest",'La mort aux trousses','Du silence et des ombres...','Chinatown','Autant en emporte le vent','Mon voisin Totoro',"Le Mans '66",'La chasse','Before Sunrise',"Lawrence d'Arabie",'Hatchi',"Certains l'aiment chaud","L'Arnaque",'Before Sunset','Le tombeau des lucioles','...et pour quelques dollars de plus','Le Bateau','Cinéma Paradiso','La grande évasion','Amours chiennes','Elephant Man','Une séparation','Ben-Hur','Chantons sous la pluie','Les temps modernes','Le pont de la rivière Kwaï','Le dictateur','Boulevard du Crépuscule','Dans ses yeux','Your Name.','Les sentiers de la gloire','Les Nouveaux Sauvages','Taare Zameen Par','Les lumières de la ville','Mary et Max.','Le septième sceau','La garçonnière','Luke la main froide','Metropolis','Le troisième homme','Le crime était presque parfait','Au nom du père','Dangal','Rashomon','Nausicaä de la vallée du vent','La Haine','Barry Lyndon','Incendies','Voleur de bicyclette','Network - Main basse sur la TV','Assurance sur la mort','M le maudit','Sur les quais...',"L'accusé",'Memories of Murder','Rebecca','In the Mood for Love','Ève...','Stalker','Le Trésor de la Sierra Madre','Le Kid','Ran','Rang De Basanti','Mademoiselle','Le garde du corps','Témoin à charge','Mr. Smith au sénat','Les quatre cents coups','Persona',"La ruée vers l'or",'Les fraises sauvages','New York - Miami','Paris, Texas','La légende de Klaus','Gangs of Wasseypur','Le mécano de la Générale','Mon père et mon fils','Andhadhun','Jugement à Nuremberg','Drishyam','Vivre','Les enfants du ciel','Eskiya le bandit','Capernaum','Portrait de la jeune fille en feu','Requiem pour un massacre - Va et regarde','Le salaire de la peur',"La bataille d'Alger",'Voyage à Tokyo','Hamilton',"La passion de Jeanne d'Arc",'Andreï Roublev','Sherlock Junior','Harakiri','Mandarines',"Entre le ciel et l'enfer",'7. Kogustaki Mucize','Anand','Jeux dangereux','Du rififi chez les hommes','Vikram Vedha',"Sonate d'automne",'Le temps des gitans']
# https://www.senscritique.com/films/tops/top100-des-top10
list4 = ['Fight Club','Pulp Fiction','Interstellar',"2001 : L'Odyssée de l'espace",'Blade Runner','Le Parrain','Forrest Gump','Le Seigneur des Anneaux : Le Retour du roi','The Dark Knight - Le Chevalier noir','Le Bon, la Brute et le Truand','Inception',"Le Seigneur des Anneaux : La Communauté de l'anneau",'Orange mécanique','Apocalypse Now','Eternal Sunshine of the Spotless Mind','Princesse Mononoké','Old Boy',"L'Empire contre-attaque",'Mulholland Drive','Matrix','Le Voyage de Chihiro','12 hommes en colère','Retour vers le futur','Il était une fois en Amérique',"Vol au-dessus d'un nid de coucou",'Into the Wild','Shining','Gladiator','Seven','Les Affranchis','Drive','Les Évadés','The Big Lebowski',"Il était une fois dans l'Ouest",'Alien : Le 8ème Passager','La Ligne verte','Django Unchained','American Beauty','Reservoir Dogs','Barry Lyndon','Shutter Island','Mommy','Le Seigneur des Anneaux : Les Deux Tours','Kill Bill : Volume 1','Jurassic Park','Taxi Driver','Donnie Darko','Usual Suspects','Requiem for a Dream','American History X','Brazil','V pour Vendetta','Inglourious Basterds',"Edward aux mains d'argent",'La Guerre des Étoiles','Snatch, tu braques ou tu raques','La vie est belle',"Le Fabuleux Destin d'Amélie Poulain",'La Liste de Schindler','The Truman Show','Le Parrain : 2e Partie','La La Land','Lost in Translation','Le Tombeau des lucioles','Her','Big Fish','Bienvenue à Gattaca','Mr. Nobody','Titanic','Memento','Le Prestige','Gran Torino','Monty Python : Sacré Graal !','Les Sept Samouraïs','Le Château ambulant','Stalker','Whiplash','Mad Max : Fury Road',"Voyage au bout de l'enfer",'Sueurs froides','Le Cercle des poètes disparus','Léon','La Cité de la peur','The Grand Budapest Hotel','Psychose','Le Silence des agneaux','Le Roi Lion','Full Metal Jacket','Trainspotting','Black Swan','Terminator 2 : Le Jugement dernier','La Ligne rouge','Heat','The Thing','Cloud Atlas','La Haine','Le Cinquième Élément',"L'Armée des 12 singes",'The Tree of Life',"Les Fils de l'homme"]
# https://www.senscritique.com/top/resultats/Les_meilleurs_films_francais/429176
list5 = ["Haine","L'Armée des ombres","Le Fabuleux Destin d'Amélie Poulain","Léon","Les Tontons flingueurs","Le Trou","La Cité de la peur","La Grande Vadrouille","Le Dîner de cons","Astérix &amp; Obélix : Mission Cléopâtre","Les Enfants du paradis","Un prophète","Les Quatre Cents Coups","Les Diaboliques","Intouchables","Le Salaire de la peur","La Grande Illusion","Le Roi et l'Oiseau","Pierrot le Fou","Le Cercle rouge","Buffet froid","La Jetée","Polisse","Cyrano de Bergerac","La Vérité","Le Samouraï","Le Corbeau","À bout de souffle","OSS 117 : Le Caire, nid d'espions","Un singe en hiver","OSS 117 : Rio ne répond plus","Irréversible","La Règle du jeu","La Belle et la Bête","La Vie d'Adèle : Chapitres 1 et 2","Garde à vue","Les Valseuses","Le Vieux Fusil","Les Visiteurs","La Traversée de Paris","Le Mépris","La Maman et la Putain","Mon oncle","Holy Motors","Série noire","Playtime","Les Demoiselles de Rochefort","Delicatessen","Le Père Noël est une ordure","Grave","La Cité des enfants perdus","Le Cinquième Élément","Les Bronzés font du ski","Jeux interdits","Les Parapluies de Cherbourg","Réalité","La Classe américaine","Bernie","Au revoir là-haut","Le Grand Bleu","Mauvais sang","De battre mon cœur s'est arrêté","La Nuit américaine","Les Trois Frères","Le Voyage dans la Lune","Hiroshima mon amour","Le Nom de la rose","L'Auberge espagnole","Quai des Orfèvres","Seul contre tous","L'assassin habite au 21","The Artist","99 Francs","Les Aventures de Rabbi Jacob","Le jour se lève","Un air de famille","Le Prénom","Enter the Void","La Folie des grandeurs","Tchao Pantin","Plein soleil","I... comme Icare","Le Deuxième Souffle","Un condamné à mort s'est échappé","Persepolis","Ernest et Célestine","Le Quai des brumes","Mesrine : 1ère Partie - L'Instinct de mort","Le Locataire","Le Clan des Siciliens","Le Premier Jour du reste de ta vie","Ma nuit chez Maud","Jules et Jim","Les Yeux sans visage","Le Pianiste","De rouille et d'os","L'Année dernière à Marienbad","Au revoir les enfants","Les Misérables","Les Douze Travaux d'Astérix"]
# https://www.senscritique.com/top/resultats/Les_films_qui_rendent_heureux/192812
list6 = ["Little Miss Sunshine","Le Fabuleux Destin d'Amélie Poulain","Good Morning England","Chantons sous la pluie","La Vie rêvée de Walter Mitty","Forrest Gump","Mon voisin Totoro","La vie est belle","Moonrise Kingdom","Un jour sans fin","Love Actually","The Grand Budapest Hotel","Retour vers le futur","Intouchables","Les Blues Brothers","Nos jours heureux","Big Fish","Certains l'aiment chaud !","Monty Python : Sacré Graal !","Astérix &amp; Obélix : Mission Cléopâtre","Mary Poppins","The Big Lebowski","Yes Man","Il était temps","La Cité de la peur","Fantastic Mr. Fox","La Grande Aventure LEGO","Happiness Therapy","Juno","La Folle Journée de Ferris Bueller","Les Gardiens de la galaxie","La La Land","(500) jours ensemble","Les Demoiselles de Rochefort","Scott Pilgrim","Rasta Rockett","Starbuck","Ponyo sur la falaise","La vie est belle","Kiki la petite sorcière","Mamma Mia !, le film","Le Monde de Charlie","Là-haut","Pride","O'Brother","Breakfast Club","Madame Doubtfire","L'Auberge espagnole","Monty Python : La Vie de Brian","La Grande Vadrouille","À bord du Darjeeling Limited","La Mélodie du bonheur","Dragons","Grease","Le Dîner de cons","Chat noir, chat blanc","Les Temps modernes","Le Nom des gens","The Mask","The Truman Show","Garden State","The Full Monty : Le Grand Jeu","L'Été de Kikujiro","La Classe américaine","Le Journal de Bridget Jones","La Vie aquatique","The Rocky Horror Picture Show","Boyhood","New York Melody","Le Voyage de Chihiro","Charlie et la Chocolaterie","Wall-E","Princess Bride","The Hit Girls (Pitch Perfect)","Eternal Sunshine of the Spotless Mind","Hairspray","Ratatouille","À la recherche du bonheur","La Reine des Neiges","Les Aventures de Rabbi Jacob","Frances Ha","S.O.S. Fantômes","Rendez-vous","Jeux dangereux","Billy Elliot","Qui veut la peau de Roger Rabbit","Vous ne l'emporterez pas avec vous","Sing Street","Diamants sur canapé","Vice-Versa","Raiponce","Le Château ambulant","Les Lumières de la ville","Le Livre de la jungle","Aladdin","L'Impossible Monsieur Bébé","Les Goonies","Captain Fantastic","Retour vers le futur 2","Coup de foudre à Notting Hill"]
# https://www.senscritique.com/top/resultats/Les_meilleurs_films_d_horreur/188963
list7 = ["Shining","L'Exorciste","The Thing","Alien : Le 8ème Passager","Psychose","Massacre à la tronçonneuse","Conjuring : Les Dossiers Warren","[REC]","La Nuit des masques","Rosemary's Baby","Suspiria","Le Projet Blair Witch","Ring","Saw","Scream","The Descent","Evil Dead","Les Griffes de la nuit","Insidious","La Mouche","Le Silence des agneaux","Les Dents de la mer","It Follows","La colline a des yeux","Les Autres","L'Antre de la folie","28 jours plus tard","Sinister","Evil Dead 2","Le Cercle : The Ring","Carrie au bal du diable","Conjuring 2 : Le Cas Enfield","La Nuit des morts-vivants","Zombie - Le Crépuscule des morts-vivants","Martyrs","Nosferatu le vampire","Les Oiseaux","L'Échelle de Jacob","The Grudge","Braindead","« Il » est revenu","Hérédité","Dark Water","2 sœurs","Haute tension","L'Orphelinat","The Devil's Rejects","Grave","Silent Hill","Paranormal Activity","Insidious : Chapitre 2","Morse","La Maison du diable","Ça","Get Out","La Cabane dans les bois","Prince des ténèbres","Esther","L'Armée des morts","Mister Babadook","Jusqu'en enfer","The Witch","Midsommar","Eden Lake","Mama","Hellraiser : Le Pacte","Sixième Sens","Evil Dead","Le Locataire","Poltergeist","Dernier train pour Busan","Halloween","Les Innocents","The Strangers","La Malédiction","Misery","Hostel","Vendredi 13","La Maison des 1000 morts","La Dame en noir","Ju-on: The Grudge","Aliens : Le Retour","Cannibal Holocaust","28 semaines plus tard","Dracula","Le Loup-garou de Londres","Candyman","Sleepy Hollow, la légende du cavalier sans tête","Jeu d'enfant","Les Frissons de l'angoisse","Shaun of the Dead","Massacre à la tronçonneuse","Vidéodrome","Grave Encounters","Eraserhead","Kaïro","Amityville, la maison du diable","[REC]²","The Visit","Les Yeux sans visage"]
# https://www.senscritique.com/top/resultats/Les_meilleurs_films_de_science_fiction/272590
list8 = ["Blade Runner","Alien : Le 8ème Passager","2001 : L'Odyssée de l'espace","Matrix","Interstellar","L'Empire contre-attaque","Bienvenue à Gattaca","Retour vers le futur","La Guerre des Étoiles","L'Armée des 12 singes","Inception","Terminator 2 : Le Jugement dernier","Brazil","Les Fils de l'homme","La Planète des singes","The Thing","Le Cinquième Élément","Metropolis","District 9","Premier contact","Minority Report","Her","Le Retour du Jedi","Ghost in the Shell","Akira","Starship Troopers","Cloud Atlas","Terminator","Jurassic Park","Blade Runner 2049","Avatar","Aliens : Le Retour","Retour vers le futur 2","Total Recall","Soleil vert","Star Wars : Épisode III - La Revanche des Sith","Orange mécanique","E.T. l'Extra-terrestre","Moon","Wall-E","La Jetée","Stalker","The Truman Show","Solaris","Rencontres du troisième type","Dark City","Gravity","Snowpiercer – le Transperceneige","Sunshine","Mad Max : Fury Road","Eternal Sunshine of the Spotless Mind","RoboCop","Donnie Darko","A.I. Intelligence Artificielle","Star Wars : Épisode I - La Menace fantôme","La Mouche","V pour Vendetta","Dune","New York 1997","Nausicaä de la vallée du vent","Edge of Tomorrow","Ex Machina","La Guerre des mondes","Star Trek","Looper","Alien 3","Abyss","Star Wars : Épisode II - L'Attaque des clones","Le Voyage dans la Lune","The Man from Earth","Retour vers le futur 3","Contact","Matrix Reloaded","Prometheus","La Planète des singes : les origines","Star Trek Into Darkness","Predator","Les Gardiens de la galaxie","Alien : La Résurrection","Paprika","Planète interdite","Oblivion","Star Wars : Le Réveil de la Force","Invasion Los Angeles","Seul sur Mars","Under the Skin","Stargate, la porte des étoiles","Matrix Revolutions","Mad Max 2 : Le Défi","Mars Attacks !","Ready Player One","Pacific Rim","Rogue One : A Star Wars Story","Vidéodrome","I, Robot","Le Jour où la Terre s'arrêta","Mr. Nobody","Equilibrium","THX 1138","Avalon"]
# https://www.senscritique.com/top/resultats/Les_meilleures_comedies/337551
list9 = ["Monty Python : Sacré Graal !","La Cité de la peur","Astérix &amp; Obélix : Mission Cléopâtre","Le Dîner de cons","The Big Lebowski","OSS 117 : Le Caire, nid d'espions","OSS 117 : Rio ne répond plus","Monty Python : La Vie de Brian","Les Tontons flingueurs","Les Visiteurs","Shaun of the Dead","La Grande Vadrouille","La Classe américaine","Hot Fuzz","Le Père Noël est une ordure","Les Bronzés font du ski","Les Trois Frères","Certains l'aiment chaud !","Le Dictateur","Les Temps modernes","The Mask","Un jour sans fin","Docteur Folamour","Intouchables","Y a-t-il un pilote dans l'avion ?","RRRrrrr !!!","Monty Python : Le Sens de la vie","C'est arrivé près de chez vous","Dikkenek","Les Aventures de Rabbi Jacob","Jeux dangereux","La Tour Montparnasse infernale","The Grand Budapest Hotel","La Folie des grandeurs","Very Bad Trip","Le Prénom","La Party","Les Blues Brothers","Dumb and Dumber","O'Brother","Tonnerre sous les tropiques","Hot Shots !","Un poisson nommé Wanda","Mary à tout prix","Hot Shots ! 2","Le Dernier pub avant la fin du monde","L'Impossible Monsieur Bébé","Bernie","Le Mécano de la « General »","Les Bronzés","Le Corniaud","Little Miss Sunshine","Buffet froid","La Chèvre","Borat, leçons culturelles sur l'Amérique au profit glorieuse nation Kazakhstan","SuperGrave","Ace Ventura, détective chiens et chats","L'Aile ou la Cuisse","Scott Pilgrim","Wayne's World","Fous d'Irène","Clerks, les employés modèles","La Légende de Ron Burgundy - Présentateur vedette","Good Morning England","Retour vers le futur","Y a-t-il un flic pour sauver la reine ?","Annie Hall","Oscar","Mais où est donc passée la 7ème compagnie...","Bienvenue à Zombieland","Papy fait de la résistance","Ace Ventura en Afrique","Sherlock Junior","Mon oncle","Qui veut la peau de Roger Rabbit","Braindead","Frankenstein Junior","Nos jours heureux","Burn After Reading","Starbuck","Scary Movie","Zoolander","9 mois ferme","Chantons sous la pluie","Snatch, tu braques ou tu raques","Tucker &amp; Dale fightent le mal","Un air de famille","La Ruée vers l'or","C'est la fin","Joyeuses Funérailles","Les Barbouzes","Le Loup de Wall Street","Arsenic et Vieilles Dentelles","Top Secret !","Las Vegas Parano","Légendes vivantes","American Pie","La Soupe aux choux","Mars Attacks !","M.A.S.H."]
# https://www.senscritique.com/top/resultats/Les_meilleurs_films_d_animation_Disney/429194
list10 = ["Le Roi Lion","Aladdin","Mulan","Le Livre de la jungle","La Belle et la Bête","Les Aristochats","Robin des bois","Alice au pays des merveilles","Peter Pan","Tarzan","Pocahontas : Une légende indienne","Merlin l'Enchanteur","Raiponce","Hercule","Blanche-Neige et les Sept Nains","Fantasia","Le Bossu de Notre-Dame","La Petite Sirène","Zootopie","Kuzco, l'empereur mégalo","La Reine des Neiges","Bambi","Pinocchio","La Belle et le Clochard","La Belle au bois dormant","Rox et Rouky","Les 101 Dalmatiens","Cendrillon","Dumbo","La Planète au trésor, un nouvel univers","Lilo &amp; Stitch","Atlantide, l'empire perdu","Basil, détective privé","Frère des ours","Les Aventures de Bernard et Bianca","Les Nouveaux Héros","Vaiana, la légende du bout du monde","La Princesse et la Grenouille","Les Mondes de Ralph","Le Roi Lion 2 : L'Honneur de la tribu","Taram et le Chaudron magique","Fantasia 2000","Rebelle","Oliver et Compagnie","Bernard et Bianca au pays des kangourous","La Bande à Picsou, le film : Le Trésor de la lampe perdue","Le Roi Lion 3 : Hakuna matata","Le Noël de Mickey","Aladdin et le Roi des voleurs","Dingo et Max"]


### Get list of imdb id automatically from imdb

In [ ]:
# TOP 1000
list_movie_imdb_id = []
url = 'https://www.imdb.com/search/title/?groups=top_1000&view=simple&sort=boxoffice_gross_us,desc'
# url = 'https://www.imdb.com/search/title/?groups=top_1000&view=simple'
for i in range(20):
    start = i*50+1
    url2 = url + f'&start={start}'
    req = requests.get(url2)
    soup = BeautifulSoup(req.text, "lxml")

    l = [re.search('tt[0-9]*',s.a['href'])[0] for s in soup.find_all('div', {'class':'col-title'})]
    list_movie_imdb_id += l

In [ ]:
# TOP FR
list_movie_imdb_id2 = []
url = 'https://www.imdb.com/search/title/?title_type=feature&country_of_origin=fr&sort=num_votes,desc&count=250&view=simple'
for i in range(4):
    start = i*250+1
    url2 = url + f'&start={start}'
    req = requests.get(url2)
    soup = BeautifulSoup(req.text, "lxml")

    l = [re.search('tt[0-9]*',s.a['href'])[0] for s in soup.find_all('div', {'class':'col-title'})]
    list_movie_imdb_id2 += l
    print(l)

In [ ]:
# TOP BY GENRE
list_genre = ["action","adventure","animation","biography","comedy","crime","drama","family","fantasy","film_noir","history","horror","music","musical","mystery","romance","sci_fi","sport","thriller","war","western"]
for genre in list_genre:
    print(genre)
    for i in range(2):
        url = f'https://www.imdb.com/search/title/?title_type=feature&genres={genre}&view=simple&sort=num_votes,desc&count=250'
        start = i*250+1
        url2 = url + f'&start={start}'
        req = requests.get(url2)
        soup = BeautifulSoup(req.text, "lxml")

        l = [re.search('tt[0-9]*',s.a['href'])[0] for s in soup.find_all('div', {'class':'col-title'})]
#         print(l)
        list_movie_imdb_id2 += l

## Get data from a movie title or imdb ID

Using BeautifulSoup you can get all the information available on the page

In [ ]:
list_df = []
DIRECT_IMDB_ID = True

for movie_id in list_movie_imdb_id:
    list_file_img = [filename for filename in [files for root, dirs, files in os.walk(FOLDER_IMG)]][0]
    
    if not DIRECT_IMDB_ID:
        print(movie_title)
        formatted_movie_title = urllib.parse.quote(movie_title)
        url = f"https://www.imdb.com/find?q={formatted_movie_title}&s=tt&ttype=ft&ref_=fn_ft"

        req = requests.get(url)
        soup = BeautifulSoup(req.text, "lxml")
        tmp_id = soup.find_all("td", {"class": "result_text"})
        if len(tmp_id) != 0:
            movie_id = tmp_id[0].find('a')['href'][7:-1]
        else:
            continue
    
    if f'{movie_id}.jpg' in list_file_img:
        continue

    url_movie = f'https://www.imdb.com/title/{movie_id}/'
    req = requests.get(url_movie)
    soup = BeautifulSoup(req.text, "lxml")
    
    movie_title_imdb = soup.find('div', {'class':'title_wrapper'}).find('h1').get_text().split('\xa0')[0]
    
    if DIRECT_IMDB_ID:
        print(movie_id, movie_title_imdb)
    tmp_rating = soup.find('span', {'itemprop':'ratingCount'})
    if tmp_rating != None:
        rating_count = int(tmp_rating.get_text().replace(',', ''))
    else:
        continue
    title_year = int(soup.find('span', {'id':'titleYear'}).find('a').get_text())
    director = soup.find('div', {'class':'credit_summary_item'}).find('a').get_text()
    image_url = soup.find('div', {'class':'poster'}).find('img')['src'].split('_V1_')[0] + '_V1_FMjpg_UY864_.jpg'
    list_tmp = soup.find('div', {'class':'title_wrapper'}).find('div', {'class':'subtext'}).find_all('a')
    if len(list_tmp) > 1:
        list_genre = [el.get_text() for el in list_tmp[:-1]]
    else:
        list_genre = []
    tmp_list_country = soup.find('div', {'id':'titleDetails'}).find(text=re.compile('Country')).parent.parent.find_all('a')
    if len(tmp_list_country) > 0:
        list_country = [el.get_text() for el in tmp_list_country]
    else:
        list_country = []

        
    # Download image
    urllib.request.urlretrieve(image_url, f'{os.path.join(FOLDER_IMG, movie_id)}.jpg')

    list_df.append([movie_id, movie_title_imdb, rating_count, title_year, director, ','.join(list_genre), ','.join(list_country)])
df = pd.DataFrame(list_df, columns=['movie_id', 'movie_title', 'rating_count', 'title_year', 'director', 'list_genre', 'list_country'])

# Save csv file
dt = datetime.now().strftime("%Y%m%d%H%M%S")
file_name = f'save_{dt}.csv'
df.to_csv(os.path.join(FOLDER_CSV, file_name), sep=';', index=False)

### From csv file, add country

In [ ]:
df_save_final = pd.read_csv(os.path.join(FOLDER_CSV, 'final', 'save.csv'), sep=';', index_col='movie_id')
df_save_final2 = df_save_final.copy()

list_tot_country = []
for movie_id, row in df_save_final.iterrows():
    url_movie = f'https://www.imdb.com/title/{movie_id}/'
    req = requests.get(url_movie)
    soup = BeautifulSoup(req.text, "lxml")
    
    tmp_list_country = soup.find('div', {'id':'titleDetails'}).find(text=re.compile('Country')).parent.parent.find_all('a')
    list_country = [c.get_text() for c in tmp_list_country]
    
    list_tot_country.append(','.join(list_country))
    print(movie_id, ','.join(list_country))
    
df_save_final2['list_country'] = list_tot_country
df_save_final2 = df_save_final2[~df_save_final2.index.duplicated(keep='first')]
df_save_final2.to_csv(os.path.join(FOLDER_CSV, 'final', 'save2.csv'), sep=';')